# Async IO

Viri:
- [Async IO in Python: A Complete Walkthrough](https://realpython.com/async-io-python/) [X]
- [How does asyncio actually work?](https://stackoverflow.com/questions/49005651/how-does-asyncio-actually-work/51116910#51116910)
- [Demystifying Python's Async and Await Keywords -> VIDEO](https://www.youtube.com/watch?v=F19R_M4Nay4) [X]
- [How the heck does async/await work in Python 3.5?](https://snarky.ca/how-the-heck-does-async-await-work-in-python-3-5/)
- [Async Techniques and Examples in Python](https://training.talkpython.fm/courses/explore_async_python/async-in-python-with-threading-and-multiprocessing) [X]
- [A Web Crawler With asyncio Coroutines](http://aosabook.org/en/500L/a-web-crawler-with-asyncio-coroutines.html)
- [Some thoughts on asynchronous API design in a post-async/await world](https://vorpus.org/blog/some-thoughts-on-asynchronous-api-design-in-a-post-asyncawait-world/)
- [Basic ideas of Python 3 asyncio concurrency](http://www.artificialworlds.net/blog/2017/05/31/basic-ideas-of-python-3-asyncio-concurrency/)
- [AsyncIO for the Working Python Developer](https://yeray.dev/python/asyncio/asyncio-for-the-working-python-developer)
- [Keynote David Beazley - Topics of Interest (Python Asyncio) -VIDEO](https://www.youtube.com/watch?v=ZzfHjytDceU&feature=youtu.be)
- [John Reese - Thinking Outside the GIL with AsyncIO and Multiprocessing - PyCon 2018](https://www.youtube.com/watch?v=0kXaLh8Fz3k&feature=youtu.be&t=10m30s) [X]
- [Lynn Root - Advanced asyncio: Solving Real-world Production Problems - PyCon 2019](https://www.youtube.com/watch?v=bckD_GK80oY)
- [AsyncIO for the Working Python Developer](https://yeray.dev/python/asyncio/asyncio-for-the-working-python-developer)
- [Waiting in asyncio](https://hynek.me/articles/waiting-in-asyncio/)
- [asyncio: We Did It Wrong](https://www.roguelynn.com/words/asyncio-we-did-it-wrong/)
- [import asyncio: Learn Python's AsyncIO #1 - The Async Ecosystem](https://www.youtube.com/watch?v=Xbl7XjFYsN4&feature=youtu.be)

Async IO is a concurrent programming design that has received dedicated support in Python, **evolving rapidly from Python 3.4 through 3.7**, and probably beyond.

**Coroutines (specialized generator functions) are the heart of async IO in Python.**

Python 3.5 and later offer three kinds of coroutines:
- `Native coroutine`: A coroutine function defined with async def. You can delegate from a native coroutine to another native coroutine using the await keyword, similar to how classic coroutines use yield from. The async def statement always defines a native coroutine, even if the await keyword is not used in its body. The await keyword cannot be used outside of a native coroutine.
- `Classic coroutine`: A generator function that consumes data sent to it via my_coro.send(data) calls, and reads that data by using yield in an expression. Classic coroutines can delegate to other classic coroutines using yield from. Classic coroutines cannot be driven by await, and are no longer supported by asyncio.
- `Generator-based coroutine`: A generator function decorated with `@types.coroutine` introduced in Python 3.5. That decorator makes the generator compatible with the new await keyword.
- `Asynchronous generator`: A generator function defined with async def and using yield in its body. It returns an asynchronous generator object that provides `__anext__`, a coroutine method to retrieve the next item.

## Introduction to Async IO

### Where Does Async IO Fit In?

Concurrency and parallelism are expansive subjects that are not easy to wade into. While this article focuses on async IO and its implementation in Python, it’s worth taking a minute to compare async IO to its counterparts in order to have context about how async IO fits into the larger, sometimes dizzying puzzle.

**Parallelism consists of performing multiple operations at the same time**. Multiprocessing is a means to effect parallelism, and it entails spreading tasks over a computer’s central processing units (CPUs, or cores). Multiprocessing is well-suited for CPU-bound tasks: tightly bound for loops and mathematical computations usually fall into this category.

Concurrency is a slightly broader term than parallelism. It suggests that multiple tasks have the ability to run in an overlapping manner. (There’s a saying that concurrency does not imply parallelism.)

Threading is a concurrent execution model whereby multiple threads take turns executing tasks. One process can contain multiple threads. Python has a complicated relationship with threading thanks to its GIL, but that’s beyond the scope of this article.

Over the last few years, a separate design has been more comprehensively built into CPython: **asynchronous IO**, enabled through the standard **library’s asyncio package** and the new **async and await language keywords**. To be clear, async IO is not a newly invented concept, and it has existed or is being built into other languages and runtime environments, such as Go, C#, or Scala.

The asyncio package is billed by the Python documentation as a library to write concurrent code. However, async IO is not threading, nor is it multiprocessing. It is not built on top of either of these.

In fact, async IO is a **single-threaded, single-process design**: it uses **cooperative multitasking**, a term that you’ll flesh out by the end of this tutorial. It has been said in other words that async IO gives a feeling of concurrency despite using a single thread in a single process. **Coroutines (a central feature of async IO) can be scheduled concurrently, but they are not inherently concurrent.**

To reiterate, async IO is a style of concurrent programming, but it is not parallelism. It’s more closely aligned with threading than with multiprocessing but is very much distinct from both of these and is a standalone member in concurrency’s bag of tricks.

That leaves one more term. What does it mean for something to be asynchronous? This isn’t a rigorous definition, but for our purposes here, I can think of two properties:
- Asynchronous routines are **able to “pause” while waiting on their ultimate result** and **let other routines run** in the meantime.
- Asynchronous code, through the mechanism above, **facilitates concurrent execution**. To put it differently, asynchronous code gives the look and feel of concurrency.

### Async IO Explained

Async IO may at first seem counterintuitive and paradoxical. How does something that facilitates concurrent code use a single thread and a single CPU core? I’ve never been very good at conjuring up examples, so I’d like to paraphrase one from Miguel Grinberg’s 2017 PyCon talk, which explains everything quite beautifully:

<blockquote>
<p>Chess master Judit Polgár hosts a chess exhibition in which she plays multiple amateur players. She has two ways of conducting the exhibition: synchronously and asynchronously.</p>
<p>Assumptions:</p>
<ul>
<li>24 opponents</li>
<li>Judit makes each chess move in 5 seconds</li>
<li>Opponents each take 55 seconds to make a move</li>
<li>Games average 30 pair-moves (60 moves total)</li>
</ul>
<p><strong>Synchronous version</strong>: Judit plays one game at a time, never two at the same time, until the game is complete. Each game takes <em>(55 + 5) * 30 == 1800</em> seconds, or 30 minutes. The entire exhibition takes <em>24 * 30 == 720</em> minutes, or <strong>12 hours</strong>.</p>
<p><strong>Asynchronous version</strong>: Judit moves from table to table, making one move at each table. She leaves the table and lets the opponent make their next move during the wait time. One move on all 24 games takes Judit <em>24 * 5 == 120</em> seconds, or 2 minutes. The entire exhibition is now cut down to <em>120 * 30 == 3600</em> seconds, or just <strong>1 hour</strong>. <a href="https://youtu.be/iG6fr81xHKA?t=4m29s">(Source)</a></p>
</blockquote>

There is only one Judit Polgár, who has only two hands and makes only one move at a time by herself. But playing asynchronously cuts the exhibition time down from 12 hours to one. So, **cooperative multitasking is a fancy way of saying that a program’s event loop (more on that later) communicates with multiple tasks to let each take turns running at the optimal time.**

Async IO takes long waiting periods in which functions would otherwise be blocking and allows other functions to run during that downtime. (**A function that blocks effectively forbids others from running** from the time that it starts until the time that it returns.)

<img loading="lazy" class="img-fluid mx-auto d-block w-100" src="https://miro.medium.com/max/1080/1*t_oCyHBstMnF8WpZ67pKTg.jpeg" sizes="75vw" alt="Timing Diagram of an I/O Bound Program">

A **synchronous program** is executed one step at a time. Even with conditional branching, loops and function calls, you can still think about the code in terms of taking one execution step at a time. When each step is complete, the program moves on to the next one.

Here are two examples of programs that work this way:
- **Batch processing** programs are often created as synchronous programs. You get some input, process it, and create some output. Steps follow one after the other until the program reaches the desired output. The program only needs to pay attention to the steps and their order.
- **Command-line** programs are small, quick processes that run in a terminal. These scripts are used to create something, transform one thing into something else, generate a report, or perhaps list out some data. This can be expressed as a series of program steps that are executed sequentially until the program is done.

An **asynchronous program** behaves differently. It still takes one execution step at a time. The difference is that the system may not wait for an execution step to be completed before moving on to the next one.

This means that the program will move on to future execution steps even though a previous step hasn’t yet finished and is still running elsewhere. This also means that the program knows what to do when a previous step does finish running.

### Blocking vs. Non-blocking code

When you start trying to understand asynchronous programming, you might see a lot of discussion about the importance of blocking, or writing non-blocking code. (Personally, I struggled to get a good grasp of these concepts from the people I asked and the documentation I read.)

What is non-blocking code? What’s blocking code, for that matter? 

Writing asynchronous programs requires that you think differently about programming. While this new way of thinking can be hard to wrap your head around, it’s also an interesting exercise. That’s because the real world is almost entirely asynchronous, and so is how you interact with it.

Imagine this: **you’re a parent trying to do several things at once**. You have to balance the checkbook, do the laundry, and keep an eye on the kids. Somehow, you’re able to do all of these things at the same time without even thinking about it! Let’s break it down:
- Balancing the checkbook is a synchronous task. One step follows another until it’s done. You’re doing all the work yourself.
- However, you can break away from the checkbook to do laundry. You unload the dryer, move clothes from the washer to the dryer, and start another load in the washer.
- Working with the washer and dryer is a synchronous task, but the bulk of the work happens after the washer and dryer are started. Once you’ve got them going, you can walk away and get back to the checkbook task. At this point, the washer and dryer tasks have become asynchronous. The washer and dryer will run independently until the buzzer goes off (notifying you that the task needs attention).
- Watching your kids is another asynchronous task. Once they are set up and playing, they can do so independently for the most part. This changes when someone needs attention, like when someone gets hungry or hurt. When one of your kids yells in alarm, you react. The kids are a long-running task with high priority. Watching them supersedes any other tasks you might be doing, like the checkbook or laundry.

These examples can help to illustrate the concepts of blocking and non-blocking code. Let’s think about this in programming terms. In this example, you’re like the CPU. **While you’re moving the laundry around, you (the CPU) are busy and blocked from doing other work, like balancing the checkbook. But that’s okay because the task is relatively quick.**

On the other hand, **starting the washer and dryer does not block you from performing other tasks**. It’s an asynchronous function because you don’t have to wait for it to finish. Once it’s started, you can go back to something else. This is called a **context switch**: the context of what you’re doing has changed, and the machine’s buzzer will notify you sometime in the future when the laundry task is complete.

As a human, this is how you work all the time. You naturally juggle multiple things at once, often without thinking about it. As a developer, the trick is how to translate this kind of behavior into code that does the same kind of thing.

### Async IO Is Not Easy

I’ve heard it said, “Use async IO when you can; use threading when you must.” The truth is that building durable multithreaded code can be hard and error-prone. Async IO avoids some of the potential speedbumps that you might otherwise encounter with a threaded design.

But that’s not to say that async IO in Python is easy. Be warned: when you venture a bit below the surface level, async programming can be difficult too! Python’s async model is built around concepts such as callbacks, events, transports, protocols, and futures—just the terminology can be intimidating. The fact that its API has been changing continually makes it no easier.

Luckily, asyncio has matured to a point where most of its features are no longer provisional, while its documentation has received a huge overhaul and some quality resources on the subject are starting to emerge as well.

## Using Python Async Features in Practice

### Synchronous Programming

This first example shows a somewhat contrived way of **having a task retrieve work from a queue and process that work**. A queue in Python is a nice FIFO (first in first out) data structure. It provides methods to put things in a queue and take them out again in the order they were inserted.

In this case, the work is to get a number from the queue and have a loop count up to that number. It prints to the console when the loop begins, and again to output the total. This program **demonstrates one way for multiple synchronous tasks to process the work in a queue.**



In [ ]:
 # example_1.py
import queue # Line 1 imports the queue module. This is where the program stores work to be done by the tasks.

# Lines 3 to 13 define task(). 
# This function pulls work out of work_queue and processes the work until there isn’t any more to do.
def task(name, work_queue):
    if work_queue.empty():
        print(f"Task {name} nothing to do")
    else:
        while not work_queue.empty():
            count = work_queue.get()
            total = 0
            print(f"Task {name} running")
            for x in range(count):
                total += 1
            print(f"Task {name} total: {total}")

# Line 15 defines main() to run the program tasks.
def main():
    """
    This is the main entry point for the program
    """
    # Create the queue of work - Line 20 creates the work_queue. All tasks use this shared resource to retrieve work.
    work_queue = queue.Queue()

    # Put some work in the queue - Lines 23 to 24 put work in work_queue. 
    # In this case, it’s just a random count of values for the tasks to process.
    for work in [15, 10, 5, 2]:
        work_queue.put(work)

    # Create some synchronous tasks
    # Line 27 creates a list of task tuples, with the parameter values those tasks will be passed.
    tasks = [(task, "One", work_queue), (task, "Two", work_queue)]

    # Run the tasks
    # Lines 30 to 31 iterate over the list of task tuples, calling each one and 
    # passing the previously defined parameter values.
    for t, n, q in tasks:
        t(n, q)

if __name__ == "__main__":
    main()

Let’s take a look at what each line does:
- Line 1 imports the queue module. This is where the program stores work to be done by the tasks.
- Lines 3 to 13 define task(). This function pulls work out of work_queue and processes the work until there isn’t any more to do.
- Line 15 defines main() to run the program tasks.
- Line 20 creates the work_queue. All tasks use this shared resource to retrieve work.
- Lines 23 to 24 put work in work_queue. In this case, it’s just a random count of values for the tasks to process.
- Line 27 creates a list of task tuples, with the parameter values those tasks will be passed.
- Lines 30 to 31 iterate over the list of task tuples, calling each one and passing the previously defined parameter values.
- Line 34 calls main() to run the program.

The task in this program is just a function accepting a string and a queue as parameters. When executed, it looks for anything in the queue to process. If there is work to do, then it pulls values off the queue, starts a for loop to count up to that value, and outputs the total at the end. It continues getting work off the queue until there is nothing left and it exits.

When this program is run, it produces the output you see below:

    Task One running
    Task One total: 15
    Task One running
    Task One total: 10
    Task One running
    Task One total: 5
    Task One running
    Task One total: 2
    Task Two nothing to do

This shows that Task One does all the work. The while loop that **Task One hits within task() consumes all the work on the queue and processes it.** When that loop exits, **Task Two gets a chance to run. However, it finds that the queue is empty**, so Task Two prints a statement that says it has nothing to do and then exits. There’s **nothing in the code to allow both Task One and Task Two to switch contexts and work together**.

### Simple Cooperative Concurrency

The next version of the program allows the two tasks to work together. **Adding a yield statement means the loop will yield control at the specified point while still maintaining its context**. This way, the yielding task can be restarted later.

The yield statement turns task() into a generator. **A generator function is called just like any other function in Python, but when the yield statement is executed, control is returned to the caller of the function. This is essentially a context switch, as control moves from the generator function to the caller**.

The interesting part is that control can be given back to the generator function by calling next() on the generator. This is a context switch back to the generator function, which picks up execution with all function variables that were defined before the yield still intact.

The while loop in main() takes advantage of this when it calls next(t). This statement restarts the task at the point where it previously yielded. All of this means that you’re in control when the context switch happens: when the yield statement is executed in task().

This is a form of cooperative multitasking. The **program is yielding control of its current context so that something else can run**. In this case, it allows the while loop in main() to run two instances of task() as a generator function. Each instance consumes work from the same queue. This is sort of clever, but it’s also a lot of work to get the same results as the first program. The program example_2.py demonstrates this simple concurrency and is listed below:

In [1]:
import queue


def task(name, queue):
    while not queue.empty():
        count = queue.get()
        total = 0
        print(f"Task {name} running")
        for x in range(count):
            total += 1
            yield
        print(f"Task {name} total: {total}")


def main():
    """
    This is the main entry point for the program
    """
    # Create the queue of work
    work_queue = queue.Queue()

    # Put some work in the queue
    for work in [15, 10, 5, 2]:
        work_queue.put(work)

    # Create some tasks
    tasks = [task("One", work_queue), task("Two", work_queue)]

    # Run the tasks
    done = False
    while not done:
        for t in tasks:
            try:
                next(t)
            except StopIteration:
                tasks.remove(t)
            if len(tasks) == 0:
                done = True


if __name__ == "__main__":
    main()


Task One running
Task Two running
Task Two total: 10
Task Two running
Task One total: 15
Task One running
Task Two total: 5
Task One total: 2


Here’s what’s happening in the code above:
- Lines 3 to 11 define task() as before, but the addition of yield on Line 10 turns the function into a generator. This where the context switch is made and control is handed back to the while loop in main().
- Line 25 creates the task list, but in a slightly different manner than you saw in the previous example code. In this case, each task is called with its parameters as its entered in the tasks list variable. This is necessary to get the task() generator function running the first time.
- Lines 31 to 36 are the modifications to the while loop in main() that allow task() to run cooperatively. This is where control returns to each instance of task() when it yields, allowing the loop to continue and run another task.
- Line 32 gives control back to task(), and continues its execution after the point where yield was called.
- Line 36 sets the done variable. The while loop ends when all tasks have been completed and removed from tasks.

This is the output produced when you run this program:

    Task One running
    Task Two running
    Task Two total: 10
    Task Two running
    Task One total: 15
    Task One running
    Task Two total: 5
    Task One total: 2

You can see that both Task One and Task Two are running and consuming work from the queue. This is what’s intended, as both tasks are processing work, and each is responsible for two items in the queue. This is interesting, but again, it takes quite a bit of work to achieve these results.

The trick here is using the yield statement, which turns task() into a generator and performs a context switch. The program uses this context switch to give control to the while loop in main(), allowing two instances of a task to run cooperatively.

Notice how Task Two outputs its total first. This might lead you to think that the tasks are running asynchronously. However, this is still a synchronous program. It’s structured so the two tasks can trade contexts back and forth. The reason why Task Two outputs its total first is that it’s only counting to 10, while Task One is counting to 15. Task Two simply arrives at its total first, so it gets to print its output to the console before Task One.

### Cooperative Concurrency With Blocking Calls

The **next version of the program is the same as the last, except for the addition of a time.sleep(delay) in the body of your task loop**. This adds a delay based on the value retrieved from the work queue to every iteration of the task loop. The delay simulates the effect of a blocking call occurring in your task.

A blocking call is code that stops the CPU from doing anything else for some period of time. In the thought experiments above, if a parent wasn’t able to break away from balancing the checkbook until it was complete, that would be a blocking call.

time.sleep(delay) does the same thing in this example, because the CPU can’t do anything else but wait for the delay to expire.

In [2]:
import time
import queue
from codetiming import Timer


def task(name, queue):
    timer = Timer(text=f"Task {name} elapsed time: {{:.1f}}")
    while not queue.empty():
        delay = queue.get()
        print(f"Task {name} running")
        timer.start()
        time.sleep(delay)
        timer.stop()
        yield


def main():
    """
    This is the main entry point for the program
    """
    # Create the queue of work
    work_queue = queue.Queue()

    # Put some work in the queue
    for work in [15, 10, 5, 2]:
        work_queue.put(work)

    tasks = [task("One", work_queue), task("Two", work_queue)]

    # Run the tasks
    done = False
    with Timer(text="\nTotal elapsed time: {:.1f}"):
        while not done:
            for t in tasks:
                try:
                    next(t)
                except StopIteration:
                    tasks.remove(t)
                if len(tasks) == 0:
                    done = True


if __name__ == "__main__":
    main()


Task One running
Task One elapsed time: 15.0
Task Two running
Task Two elapsed time: 10.0
Task One running
Task One elapsed time: 5.0
Task Two running
Task Two elapsed time: 2.0

Total elapsed time: 32.0


Here’s what’s different in the code above:
- Line 1 imports the time module to give the program access to time.sleep().
- Line 3 imports the the Timer code from the codetiming module.
- Line 6 creates the Timer instance used to measure the time taken for each iteration of the task loop.
- Line 10 starts the timer instance
- Line 11 changes task() to include a time.sleep(delay) to mimic an IO delay. This replaces the for loop that did the counting in example_1.py.
- Line 12 stops the timer instance and outputs the elapsed time since timer.start() was called.
- Line 30 creates a Timer context manager that will output the elapsed time the entire while loop took to execute.

When you run this program, you’ll see the following output:

    Task One running
    Task One elapsed time: 15.0
    Task Two running
    Task Two elapsed time: 10.0
    Task One running
    Task One elapsed time: 5.0
    Task Two running
    Task Two elapsed time: 2.0

    Total elapsed time: 32.0

As before, both Task One and Task Two are running, consuming work from the queue and processing it. However, even with the addition of the delay, you can see that cooperative concurrency hasn’t gotten you anything. **The delay stops the processing of the entire program, and the CPU just waits for the IO delay to be over.**

This is exactly what’s meant by **blocking code** in Python async documentation. You’ll notice that the time it takes to run the entire program is just the cumulative time of all the delays. Running tasks this way is not a win.

### Cooperative Concurrency With Non-Blocking Calls

The next version of the program has been modified quite a bit. It makes use of Python async features using asyncio/await provided in Python 3.

The **time and queue modules have been replaced with the asyncio package**. This gives your program access to asynchronous friendly (**non-blocking) sleep and queue functionality**. The change to task() defines it as asynchronous with the addition of the async prefix on line 4. This indicates to Python that the function will be asynchronous.

The other big change is removing the **time.sleep(delay) and yield statements, and replacing them with await asyncio.sleep(delay). This creates a non-blocking delay that will perform a context switch back to the caller main().**

The while loop inside main() no longer exists. Instead of task_array, there’s a call to await `asyncio.gather(...)`. This tells asyncio two things:
1. Create two tasks based on task() and start running them.
2. Wait for both of these to be completed before moving forward.

The last line of the program `asyncio.run(main())` runs main(). This creates what’s known as an event loop). It’s this loop that will run main(), which in turn will run the two instances of task().

The **event loop is at the heart of the Python async system**. It runs all the code, including main(). When task code is executing, the CPU is busy doing work. When the await keyword is reached, a context switch occurs, and control passes back to the event loop. The event loop looks at all the tasks waiting for an event (in this case, an asyncio.sleep(delay) timeout) and passes control to a task with an event that’s ready.

await asyncio.sleep(delay) is non-blocking in regards to the CPU. Instead of waiting for the delay to timeout, the CPU registers a sleep event on the event loop task queue and performs a context switch by passing control to the event loop. The event loop continuously looks for completed events and passes control back to the task waiting for that event. In this way, the CPU can stay busy if work is available, while the event loop monitors the events that will happen in the future.

> Note: An asynchronous program runs in a single thread of execution. The context switch from one section of code to another that would affect data is completely in your control. This means you can atomize and complete all shared memory data access before making a context switch. This simplifies the shared memory problem inherent in threaded code.

In [ ]:
import asyncio
from codetiming import Timer


async def task(name, work_queue):
    timer = Timer(text=f"Task {name} elapsed time: {{:.1f}}")
    while not work_queue.empty():
        delay = await work_queue.get()
        print(f"Task {name} running")
        timer.start()
        await asyncio.sleep(delay)
        timer.stop()


async def main():
    """
    This is the main entry point for the program
    """
    # Create the queue of work
    work_queue = asyncio.Queue()

    # Put some work in the queue
    for work in [15, 10, 5, 2]:
        await work_queue.put(work)

    tasks = [
        asyncio.create_task(task("One", work_queue)),
        asyncio.create_task(task("Two", work_queue)),
    ]

    # Run the tasks
    with Timer(text="\nTotal elapsed time: {:.1f}"):
        await asyncio.gather(*tasks)


if __name__ == "__main__":
    asyncio.run(main())

Here’s what’s different between this program and example_3.py:
- Line 1 imports asyncio to gain access to Python async functionality. This replaces the time import.
- Line 2 imports the the Timer code from the codetiming module.
- Line 4 shows the addition of the async keyword in front of the task() definition. This informs the program that task can run asynchronously.
- Line 5 creates the Timer instance used to measure the time taken for each iteration of the task loop.
- Line 9 starts the timer instance
- Line 10 replaces time.sleep(delay) with the non-blocking asyncio.sleep(delay), which also yields control (or switches contexts) back to the main event loop.
- Line 11 stops the timer instance and outputs the elapsed time since timer.start() was called.
- Line 18 creates the non-blocking asynchronous work_queue.
- Lines 21 to 22 put work into work_queue in an asynchronous manner using the await keyword.
- Line 25 creates a Timer context manager that will output the elapsed time the entire while loop took to execute.
- Lines 26 to 29 create the two tasks and gather them together, so the program will wait for both tasks to complete.
- Line 32 starts the program running asynchronously. It also starts the internal event loop.

When you look at the output of this program, notice how both Task One and Task Two start at the same time, then wait at the mock IO call:

    Task One running
    Task Two running
    Task Two total elapsed time: 10.0
    Task Two running
    Task One total elapsed time: 15.0
    Task One running
    Task Two total elapsed time: 5.0
    Task One total elapsed time: 2.0

    Total elapsed time: 17.0

This indicates that **await asyncio.sleep(delay) is non-blocking**, and that other work is being done.

At the end of the program, you’ll notice the total elapsed time is essentially half the time it took for example_3.py to run. That’s the advantage of a program that uses Python async features! Each task was able to run await asyncio.sleep(delay) at the same time. The total execution time of the program is now less than the sum of its parts. You’ve broken away from the synchronous model!

## The asyncio Package and async/await

Now that you have some background on async IO as a design, let’s explore Python’s implementation. Python’s asyncio package (introduced in Python 3.4) and its two keywords, async and await, serve different purposes but come together to help you declare, build, execute, and manage asynchronous code.

### The async/await Syntax and Native Coroutines

> A Word of Caution: Be careful what you read out there on the Internet. Python’s async IO API has evolved rapidly from Python 3.4 to Python 3.7. Some old patterns are no longer used, and some things that were at first disallowed are now allowed through new introductions. For all I know, this tutorial will join the club of the outdated soon too.

Contrast this to the synchronous version:

In [ ]:
import time


def count():
    print("One")
    time.sleep(1)
    print("Two")


def main():
    for _ in range(3):
        count()


if __name__ == "__main__":
    s = time.perf_counter()
    main()
    elapsed = time.perf_counter() - s
    print(f"{__file__} executed in {elapsed:0.2f} seconds.")

When executed, there is a slight but critical change in order and execution time:

    $ python3 countsync.py
    One
    Two
    One
    Two
    One
    Two
    countsync.py executed in 3.01 seconds.

At the heart of async IO are coroutines. A **oroutine is a specialized version of a Python generator function**. Let’s start with a baseline definition and then build off of it as you progress here: a **coroutine is a function that can suspend its execution before reaching return, and it can indirectly pass control to another coroutine for some time**.

Later, you’ll dive a lot deeper into how exactly the traditional generator is repurposed into a coroutine. For now, the easiest way to pick up how coroutines work is to start making some.

Let’s take the immersive approach and write some async IO code. This short program is the Hello World of async IO but goes a long way towards illustrating its core functionality:



> Pokažemo najprej z navadnim sleepom

In [ ]:
import asyncio
import time


async def count():
    print("One")
    await asyncio.sleep(1)
    print("Two")


async def main():
    await asyncio.gather(count(), count(), count())


if __name__ == "__main__":
    s = time.perf_counter()
    asyncio.run(main())
    elapsed = time.perf_counter() - s
    print(f"{__file__} executed in {elapsed:0.2f} seconds.")

When you execute this file, take note of what looks different than if you were to define the functions with just def and time.sleep():

    $ python3 countasync.py
    One
    One
    One
    Two
    Two
    Two
    countasync.py executed in 1.01 seconds.

The order of this output is the heart of async IO. Talking to each of the calls to count() is a single event loop, or coordinator. When each task reaches await asyncio.sleep(1), the function yells up to the event loop and gives control back to it, saying, “I’m going to be sleeping for 1 second. Go ahead and let something else meaningful be done in the meantime.”



While using time.sleep() and asyncio.sleep() may seem banal, they are used as stand-ins for any time-intensive processes that involve wait time. (The most mundane thing you can wait on is a sleep() call that does basically nothing.) That is, time.sleep() can represent any time-consuming blocking function call, while asyncio.sleep() is used to stand in for a non-blocking call (but one that also takes some time to complete).

As you’ll see in the next section, the benefit of awaiting something, including asyncio.sleep(), is that the surrounding function can temporarily cede control to another function that’s more readily able to do something immediately. In contrast, time.sleep() or any other blocking call is incompatible with asynchronous Python code, because it will stop everything in its tracks for the duration of the sleep time.

### Run a coroutine with asyncio

To actually run a coroutine, asyncio provides three main mechanisms:

- The asyncio.run() function to run the top-level entry point “main()” function (see the above example.)
- Awaiting on a coroutine. The following snippet of code will print “hello” after waiting for 1 second, and then print “world” after waiting for another 2 seconds:

In [ ]:
# run_await.py
import asyncio
import time

async def say_after(delay, what):
    await asyncio.sleep(delay)
    print(what)

async def main():
    print(f"started at {time.strftime('%X')}")

    await say_after(1, 'hello')
    await say_after(2, 'world')

    print(f"finished at {time.strftime('%X')}")

asyncio.run(main())

- The asyncio.create_task() function to run coroutines concurrently as asyncio Tasks.

Let’s modify the above example and run two say_after coroutines concurrently:

In [ ]:
# run_create_task.py

import asyncio
import time

async def say_after(delay, what):
    await asyncio.sleep(delay)
    print(what)

async def main():
    task1 = asyncio.create_task(say_after(1, 'hello'))
    task2 = asyncio.create_task(say_after(2, 'world'))

    print(f"started at {time.strftime('%X')}")

    # Wait until both tasks are completed (should take
    # around 2 seconds.)
    await task1
    await task2

    print(f"finished at {time.strftime('%X')}")
          

asyncio.run(main())

Note that expected output now shows that the snippet runs 1 second faster than before

### Awaitables

We say that an object is an **awaitable object if it can be used in an await expression**. Many asyncio APIs are designed to accept awaitables.

**There are three main types of awaitable objects: coroutines, Tasks, and Futures.**

**Coroutines**

Python coroutines are awaitables and therefore can be awaited from other coroutines:

In [ ]:
import asyncio

async def nested():
    return 42

async def main():
    # Nothing happens if we just call "nested()".
    # A coroutine object is created but not awaited,
    # so it *won't run at all*.
    print(nested())

    # Let's do it differently now and await it:
    print(await nested())  # will print "42".

asyncio.run(main())

Important In this documentation the term “coroutine” can be used for two closely related concepts:

- a coroutine function: an async def function;
- a coroutine object: an object returned by calling a coroutine function.

asyncio also supports legacy generator-based coroutines.

**Tasks**

Tasks are used to schedule coroutines concurrently.

When a coroutine is wrapped into a Task with functions like asyncio.create_task() the coroutine is automatically scheduled to run soon:

In [ ]:
import asyncio


async def nested():
    return 42


async def main():
    # Schedule nested() to run soon concurrently
    # with "main()".
    task = asyncio.create_task(nested())

    # "task" can now be used to cancel "nested()", or
    # can simply be awaited to wait until it is complete:
    print(await task)


asyncio.run(main())


> `asyncio.create_task(coro, *, name=None)` Wrap the coro coroutine into a Task and schedule its execution. Return the Task object. If name is not None, it is set as the name of the task using Task.set_name().
The task is executed in the loop returned by get_running_loop(), RuntimeError is raised if there is no running loop in current thread.
This function has been added in Python 3.7. Prior to Python 3.7, the low-level asyncio.ensure_future() function can be used instead:

**Futures**

Future objects are used to bridge low-level callback-based code with high-level async/await code.

https://docs.python.org/3/library/asyncio-future.html

A Future is a special low-level awaitable object that represents an eventual result of an asynchronous operation.

When a Future object is awaited it means that the coroutine will wait until the Future is resolved in some other place.

Future objects in asyncio are needed to allow callback-based code to be used with async/await.

Normally there is no need to create Future objects at the application level code.

## Waiting in asyncio

One of the main appeals of using Python’s asyncio is being able to fire off many coroutines and run them concurrently.

Before I start, a few definitions that I will use throughout this post:
- `coroutine`: A running asynchronous function. So if you define a function as async def f(): ... and call it as f(), you get back a coroutine in the sense that the term is used throughout this post.
- `awaitable`: anything that works with await: coroutines, asyncio.Futures, asyncio.Tasks, objects that have a `__await__` method.
- I will be using two async functions f and g for my examples. It’s not important what they do, only that they are defined as async def f(): ... and async def g(): ... and that they terminate eventually.

### await

The simplest case is to await your coroutines:

    result_f = await f()
    result_g = await g()

In [ ]:
import asyncio
import time


async def f(delay, what):
    await asyncio.sleep(delay)
    print("function f", what)
    return what


async def g(delay, what):
    await asyncio.sleep(delay)
    print("function g", what)
    return what


async def main():
    print(f"started at {time.strftime('%X')}")
    result_f = await f(2, "prva")
    result_g = await g(2, "druga")
    print(f"finished at {time.strftime('%X')}")
    print(result_f, result_g)


if __name__ == "__main__":
    asyncio.run(main())

However:
1. The coroutines **do not run concurrently**. g only starts executing after f has finished.
2. You **can’t cancel them once you started awaiting**.

A naïve approach to the first problem might be something like this:

In [ ]:
import asyncio
import time


async def f(delay, what):
    await asyncio.sleep(delay)
    print("function f", what)
    return what


async def g(delay, what):
    await asyncio.sleep(delay)
    print("function g", what)
    return what


async def main():
    print(f"started at {time.strftime('%X')}")
    coro_f = f(2, "prva")
    coro_g = g(2, "druga")
    result_f = await coro_f
    result_g = await coro_g
    print(f"finished at {time.strftime('%X')}")
    print(result_f, result_g)


if __name__ == "__main__":
    asyncio.run(main())

But the execution of `g/coro_g` doesn’t start before it is awaited, making it identical to the first example. For both problems you need to wrap your coroutines in tasks.

### Tasks

`asyncio.Task`s wrap your coroutines and get independently scheduled for execution by the event loop whenever you yield control to it (Usually by awaiting something.). You can create them using `asyncio.create_task()`:

In [ ]:
import asyncio
import time


async def f(delay, what):
    await asyncio.sleep(delay)
    print("function f", what)
    return what


async def g(delay, what):
    await asyncio.sleep(delay)
    print("function g", what)
    return what


async def main():
    print(f"started at {time.strftime('%X')}")
    #  must create both tasks before you await the first one – otherwise you gain nothing
    task_f = asyncio.create_task(f(2, "prva"))
    task_g = asyncio.create_task(g(2, "druga"))
    await asyncio.sleep(1)  # <- f() and g() are already running!
    result_f = await task_f
    result_g = await task_g
    print(f"finished at {time.strftime('%X')}")
    print(result_f, result_g)


if __name__ == "__main__":
    asyncio.run(main())

Your **tasks now run concurrently** and if you decide that you don’t want to wait for `task_f` or `task_g` to finish, you can cancel them using `task_f.cancel()` or `task_g.cancel()` respectively. Please note that you **must create both tasks before you await the first one** – otherwise you gain nothing. However, **the awaits are only needed to collect the results and to clean up resources**. asyncio will complain if you don’t consume all your results and exceptions.

But waiting for each of them like this is not very practical. **In real life code you often enough don’t even know how many awaitables** you will need to wrangle. What we need is to gather the results of multiple awaitables.

### asyncio.gather()

`asyncio.gather()` takes 1 or more awaitables as `*args`, **wraps them in tasks if necessary**, and **waits for all of them to finish**. Then it **returns the results of all awaitables in the same order as you passed in the awaitables**:

    result_f, result_g = await asyncio.gather(f(), g())

In [ ]:
import asyncio
import time


async def f(delay, what):
    await asyncio.sleep(delay)
    print("function f", what)
    return what


async def g(delay, what):
    await asyncio.sleep(delay)
    print("function g", what)
    return what


async def main():
    print(f"started at {time.strftime('%X')}")
    result = await asyncio.gather(f(2, "prva"), g(2, "druga"))
    print(f"finished at {time.strftime('%X')}")
    print(result)


if __name__ == "__main__":
    asyncio.run(main())


If f() or g() raise an exception, `gather()` will **raise it immediately**, but the other tasks are not affected. However if gather() itself is canceled, all of the awaitables that it’s gathering – and that have not completed yet – are also canceled.

You can also pass `return_exceptions=True` and then exceptions are returned like normal results and you have to check yourself whether or not they were successful (e.g. using isinstance(result, BaseException).

In [ ]:
import asyncio
import time


async def f(delay, what):
    await asyncio.sleep(delay)
    print("function f", what)
    return what


async def g(delay, what):
    await asyncio.sleep(delay)
    print("function g", what)
    return what


async def h(delay, what):
    await asyncio.sleep(delay)
    raise ValueError
    print("function h", what)
    return what


async def main():
    print(f"started at {time.strftime('%X')}")
    # pokažemo še brez return_exceptions=True
    result = await asyncio.gather(
        f(2, "prva"), g(2, "druga"), h(2, "tretja"), return_exceptions=True
    )
    print(f"finished at {time.strftime('%X')}")
    print(result)


if __name__ == "__main__":
    asyncio.run(main())

- Takes many awaitables as *args.
- Wraps each awaitable in a task if necessary.
- Returns the list of results in the same order.
    - Allows errors to be returned as results (by passing return_exceptions=True).
    - Otherwise if one of the awaitables raises an exception, gather() propagates it immediately to the caller. But the remaining tasks keep running.
- If gather() itself is canceled, it cancels all unfinished tasks it’s gathering.

Now we can wait for many awaitables at once! However well-behaved distributed systems need timeouts. Since gather() hasn’t an option for that, we need the next helper.

### asyncio.wait_for()

`asyncio.wait_for()` takes two arguments: **one awaitable and a timeout in seconds**. If the awaitable is a coroutine, it will **automatically be wrapped by a task**. So the following construct is quite common:

In [ ]:
import asyncio
import time


async def f(delay, what):
    await asyncio.sleep(delay)
    print("function f", what)
    return what


async def g(delay, what):
    await asyncio.sleep(delay)
    print("function g", what)
    return what


async def main():
    print(f"started at {time.strftime('%X')}")
    try:
        result = await asyncio.wait_for(
            asyncio.gather(f(2, "prva"), g(2, "druga")), timeout=1.5
        )
    except asyncio.TimeoutError:
        print("oops took longer than 1.5s!")
    else:
        print(result)

    print(f"finished at {time.strftime('%X')}")


if __name__ == "__main__":
    asyncio.run(main())


If the timeout expires, the inner task gets cancelled. Which for gather() means that all tasks that it is gathering are canceled too: in this case f() and g().

Please note that just replacing `create_task()` by `wait_for()` and calling it a day does not work. `create_task()` is a regular function that returns a task; `wait_for()` is an async function that returns a coroutine. That means it does not start executing until you await it:

In [ ]:
# NOT concurrent!
cf = asyncio.wait_for(f(), timeout=0.5)
cg = asyncio.wait_for(g(), timeout=0.5)

# cf and cg are both COROUTINES, not tasks!
# At THIS point, there's NOTHING to be scheduled by the event loop.

await cf  # g() is NOT executing yet!
await cg  # wait_for creates only HERE the task for g()

If you now think that there would be no need for `wait_for()` if `gather()` had a timeout option, we’re thinking the same thing.

- Takes one awaitable.
- Wraps the awaitable in a task if necessary.
- Takes a timeout that cancels the task if it expires.
- Unlike create_task(), is a coroutine itself that doesn’t execute until awaited.

A more elegant approach to timeouts is the async-timeout package on PyPI. It gives you an asynchronous context manager that allows you to apply a total timeout even if you need to execute the coroutines sequentially:

https://pypi.org/project/async-timeout/

In [ ]:
async with async_timeout.timeout(5.0):
    await f()
    await g()

Sometimes, you don’t want to wait until all awaitables are done. Maybe you want to process them as they finish and report some kind of progress to the user.

### asyncio.as_completed()

Run awaitable objects in the aws iterable concurrently. Return an iterator of coroutines. Each coroutine returned can be awaited to get the earliest next result from the iterable of the remaining awaitables.

Raises asyncio.TimeoutError if the timeout occurs before all Futures are done.

In [ ]:
for fut in asyncio.as_completed([task_f, task_g], timeout=5.0):
    try:
        await fut
        print("one task down!")
    except Exception:
        print("ouch")

There’s no way to find out which awaitable you’re awaiting though.

- Takes many awaitables in an iterable.
- Yields Futures that you have to await as soon as something is done.
- Does not guarantee to return the original awaitables that you passed in.
- Does wrap the awaitables in tasks (it actually calls asyncio.ensure_future() on them).
- Takes an optional timeout.

Finally, you may want more control over waiting and that takes us to the final waiting primitive.

### asyncio.wait()

`asyncio.wait()` is the most unwieldy of the APIs but also the most powerful one. It reminds a little of the venerable select() system call.

Like as_completed(), it **takes awaitables in an iterable**. It will **return two sets: the awaitables that are done and those that are still pending**. It’s up to you to await them and to determine which result belongs to what:

In [ ]:
done, pending = await asyncio.wait([task_f, task_g])

for t in done:
    try:
        if t is task_f:
            print(f"The result of f() is { await task_f }.")
    except Exception as e:
        print(f"f() failed with { repr(e) }.")

# ...and same for g()

## The Rules of Async IO

At this point, a more formal definition of async, await, and the coroutine functions that they create are in order. This section is a little dense, but getting a hold of async/await is instrumental, so come back to this if you need to:
- The syntax async def introduces either a **native coroutine** or an **asynchronous generator**. The expressions async with and async for are also valid, and you’ll see them later on.
- The keyword **await passes function control back to the event loop**. (It suspends the execution of the surrounding coroutine.) If Python encounters an await f() expression in the scope of g(), this is how await tells the event loop, “Suspend execution of g() until whatever I’m waiting on—the result of f()—is returned. In the meantime, go let something else run.”

In code, that second bullet point looks roughly like this:

In [5]:
async def g():
    # Pause here and come back to g() when f() is ready
    r = await f()
    return r

There’s also a strict set of rules around when and how you can and cannot use async/await. These can be handy whether you are still picking up the syntax or already have exposure to using async/await:
- A **function that you introduce with async def is a coroutine**. It may use await, return, or yield, but all of these are optional. Declaring async def noop(): pass is valid:
    - Using await and/or return creates a coroutine function. To call a coroutine function, you must await it to get its results.
    - It is less common (and only recently legal in Python) to use yield in an async def block. This creates an asynchronous generator, which you iterate over with async for. Forget about async generators for the time being and focus on getting down the syntax for coroutine functions, which use await and/or return.
    - Anything defined with async def may not use yield from, which will raise a SyntaxError.
- Just like it’s a SyntaxError to use yield outside of a def function, it is a SyntaxError to use await outside of an async def coroutine. You can only use await in the body of coroutines.

Here are some terse examples meant to summarize the above few rules:

In [ ]:
async def f(x):
    y = await z(x)  # OK - `await` and `return` allowed in coroutines
    return y

async def g(x):
    yield x  # OK - this is an async generator
    
async def m(x):
    yield from gen(x)  # No - SyntaxError
    
def m(x):
    y = await z(x)  # Still no - SyntaxError (no `async def` here)
    return y

<p>Finally, when you use <code>await f()</code>, it’s required that <code>f()</code> be an object that is <a href="https://docs.python.org/3/reference/datamodel.html#awaitable-objects">awaitable</a>. Well, that’s not very helpful, is it? For now, just know that an awaitable object is either (1) another coroutine or (2) an object defining an <code>.__await__()</code> dunder method that returns an iterator. If you’re writing a program, for the large majority of purposes, you should only need to worry about case #1.</p>

<p>That brings us to one more technical distinction that you may see pop up: an older way of marking a function as a coroutine is to decorate a normal <code>def</code> function with <code>@asyncio.coroutine</code>. The result is a <strong>generator-based coroutine</strong>. This construction has been outdated since the <code>async</code>/<code>await</code> syntax was put in place in Python 3.5.</p>

<p>These two coroutines are essentially equivalent (both are awaitable), but the first is <strong>generator-based</strong>, while the second is a <strong>native coroutine</strong>:</p>

In [ ]:
import asyncio

@asyncio.coroutine
def py34_coro():
    """Generator-based coroutine, older syntax"""
    yield from stuff()

async def py35_coro():
    """Native coroutine, modern syntax"""
    await stuff()

If you’re writing any code yourself, prefer native coroutines for the sake of being explicit rather than implicit. Generator-based coroutines will be removed in Python 3.10.



Towards the latter half of this tutorial, we’ll touch on generator-based coroutines for explanation’s sake only. **The reason that async/await were introduced is to make coroutines a standalone feature of Python that can be easily differentiated from a normal generator function**, thus reducing ambiguity.

Don’t get bogged down in generator-based coroutines, which have been deliberately outdated by async/await. They have their own small set of rules (for instance, await cannot be used in a generator-based coroutine) that are largely irrelevant if you stick to the async/await syntax.


## Example 1: Getting web data and writing to files

For our examples in this section, we're going to build a small Python program that grabs a random music genre from Binary Jazz's Genrenator API five times, prints the genre to the screen, and puts each one into its own file.

Sync version:

In [ ]:
import os
import time
from pathlib import Path

import requests


def get_absolute_file_path(relative_path: str) -> str:
    return str(Path(__file__).parent.joinpath(relative_path))


def write_genre(file_name):
    """
    Uses genrenator from binaryjazz.us to write a random genre to the
    name of the given file
    """

    req = requests.get(
        "https://binaryjazz.us/wp-json/genrenator/v1/genre/",
        headers={"User-Agent": "Mozilla/5.0"},
    )
    genre = req.json()

    with open(file_name, "w") as new_file:
        print(f"Writing '{genre}' to '{file_name}'...")
        new_file.write(genre)


if __name__ == "__main__":
    path = get_absolute_file_path("sync")
    os.makedirs(path, exist_ok=True)

    print("Starting...")
    start = time.perf_counter()

    for i in range(10):
        write_genre(f"{path}/new_file{i}.txt")

    end = time.perf_counter()
    print(f"Time to complete synchronous read/writes: {round(end - start, 2)} seconds")


Let's take a look at an example using asyncio. For this method, we're going to install aiohttp using pip. This will allow us to make non-blocking requests and receive responses using the async/await syntax that will be introduced shortly. It also has the extra benefit of a function that converts a JSON response without needing to import the json library. We'll also install and import aiofiles, which allows non-blocking file operations. Other than aiohttp and aiofiles, import asyncio, which comes with the Python standard library.

Once we have our imports in place, let's take a look at the asynchronous version of the write_genre function from our asyncio example:

In [ ]:
import asyncio
import os
import sys
import time
from pathlib import Path

import aiofiles
import aiohttp


def get_absolute_file_path(relative_path: str) -> str:
    return str(Path(__file__).parent.joinpath(relative_path))


async def write_genre(file_name):
    """
    Uses genrenator from binaryjazz.us to write a random genre to the
    name of the given file
    """

    async with aiohttp.ClientSession() as session:
        async with session.get(
            "https://binaryjazz.us/wp-json/genrenator/v1/genre/"
        ) as response:
            genre = await response.json()

    async with aiofiles.open(file_name, "w") as new_file:
        print(f"Writing '{genre}' to '{file_name}'...")
        await new_file.write(genre)


async def main():
    path = get_absolute_file_path("async")
    os.makedirs(path, exist_ok=True)

    tasks = []

    print("Starting...")
    start = time.time()

    for i in range(10):
        tasks.append(write_genre(f"{path}/new_file{i}.txt"))

    await asyncio.gather(*tasks)

    end = time.time()
    print(f"Time to complete asyncio read/writes: {round(end - start, 2)} seconds")


if __name__ == "__main__":
    # On Windows, this finishes successfully, but throws 'RuntimeError: Event loop is closed'
    # The following lines fix this
    # Source: https://github.com/encode/httpx/issues/914#issuecomment-622586610
    if (
        sys.version_info[0] == 3
        and sys.version_info[1] >= 8
        and sys.platform.startswith("win")
    ):
        asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())

    asyncio.run(main())


We're using async with to open our client session asynchronously. The aiohttp.ClientSession() class is what allows us to make HTTP requests and remain connected to a source without blocking the execution of our code. We then make an async request to the Genrenator API and await the JSON response (a random music genre). In the next line, we use async with again with the aiofiles library to asynchronously open a new file to write our new genre to. We print the genre, then write it to the file.

Unlike regular Python scripts, programming with asyncio pretty much enforces* using some sort of "main" function.

This is because you need to use the "async" keyword in order to use the "await" syntax, and the "await" syntax is the only way to actually run other async functions.

As you can see, we've declared it with "async." We then create an empty list called "tasks" to house our async tasks (calls to Genrenator and our file I/O). We append our tasks to our list, but they are not actually run yet. The calls don't actually get made until we schedule them with await asyncio.gather(*tasks). This runs all of the tasks in our list and waits for them to finish before continuing with the rest of our program. Lastly, we use asyncio.run(main()) to run our "main" function. The .run() function is the entry point for our program, and it should generally only be called once per process.

## Example 2: Simple Web Scraping

In [ ]:
import asyncio
import aiohttp
import bs4
import time
from colorama import Fore


async def get_html(episode_number: int) -> str:
    print(Fore.YELLOW + f"Getting HTML for episode {episode_number}", flush=True)

    url = f"https://talkpython.fm/{episode_number}"
    # resp = await requests.get(url) -> ta del najdlje traja zato ga damo na async, probelm da knjižnica request ni async
    async with aiohttp.ClientSession() as session:  # pridovinaje seje je asinhrono, zato moremo dodat še async
        async with session.get(url) as resp:  # asinhorn začetek http zahtevka
            resp.raise_for_status()
            html = await resp.text()  # čakanje na podatke iz srverja
            return (html, episode_number)


def get_title(html: str, episode_number: int) -> str:
    print(Fore.CYAN + f"Getting TITLE for episode {episode_number}", flush=True)
    soup = bs4.BeautifulSoup(html, "html.parser")
    header = soup.select_one("h1")
    if not header:
        return "MISSING"

    return header.text.strip()


async def get_title_range():
    # Please keep this range pretty small to not DDoS my site. ;)
    tasks = []
    for n in range(185, 200):
        tasks.append(asyncio.create_task(get_html(n)))

    results = await asyncio.gather(*tasks)

    for html, episode_number in results:
        title = get_title(
            html, episode_number
        )  # to je procesiranje v pomnilniku zato ni problematično, async ne pohitri, edina rešitev mulitprocesing
        print(Fore.WHITE + f"Title found: {title}", flush=True)


def main():
    start = time.perf_counter()
    asyncio.run(get_title_range())
    elapsed = time.perf_counter() - start
    print(f"Program completed in {elapsed:0.5f} seconds.")
    print("Done.")


if __name__ == "__main__":
    main()

## The Event Loop

In its purest essence, an event loop is a process that waits around for triggers and then performs specific (programmed) actions once those triggers are met. They often return a "promise" (JavaScript syntax) or "future" (Python syntax) of some sort to denote that a task has been added. Once the task is finished, the promise or future returns a value passed back from the called function (assuming the function does return a value).

The idea of performing a function in response to another function is called a "callback."

---

You can think of an event loop as something like a while True loop that monitors coroutines, taking feedback on what’s idle, and looking around for things that can be executed in the meantime. It is able to wake up an idle coroutine when whatever that coroutine is waiting on becomes available.

Thus far, the entire management of the event loop has been implicitly handled by one function call:

In [ ]:
asyncio.run(main())  # Python 3.7+

asyncio.run(), introduced in Python 3.7, is responsible for getting the event loop, running tasks until they are marked as complete, and then closing the event loop.

There’s a more long-winded way of managing the asyncio event loop, with get_event_loop(). The typical pattern looks like this:

In [ ]:
loop = asyncio.get_event_loop()
try:
    loop.run_until_complete(main())
finally:
    loop.close()

You’ll probably see loop.get_event_loop() floating around in older examples, but unless you have a specific need to fine-tune control over the event loop management, asyncio.run() should be sufficient for most programs.

If you do need to interact with the event loop within a Python program, loop is a good-old-fashioned Python object that supports introspection with loop.is_running() and loop.is_closed(). You can manipulate it if you need to get more fine-tuned control, such as in scheduling a callback by passing the loop as an argument.

What is more crucial is understanding a bit beneath the surface about the mechanics of the event loop. Here are a few points worth stressing about the event loop.

`#1: Coroutines don’t do much on their own until they are tied to the event loop.`

You saw this point before in the explanation on generators, but it’s worth restating. If you have a main coroutine that awaits others, simply calling it in isolation has little effect:

In [ ]:
import asyncio

async def main():
    print("Hello ...")
    await asyncio.sleep(1)
    print("World!")

    >>> routine = main()
    >>> routine
    <coroutine object main at 0x1027a6150>

Remember to use asyncio.run() to actually force execution by scheduling the main() coroutine (future object) for execution on the event loop:

    >>> asyncio.run(routine)
    Hello ...
    World!

(Other coroutines can be executed with await. It is typical to wrap just main() in asyncio.run(), and chained coroutines with await will be called from there.)

`#2: By default, an async IO event loop runs in a single thread and on a single CPU core.`

Usually, running one single-threaded event loop in one CPU core is more than sufficient. It is also possible to run event loops across multiple cores. 

`#3. Event loops are pluggable.`

That is, you could, if you really wanted, write your own event loop implementation and have it run tasks just the same. This is wonderfully demonstrated in the uvloop package, which is an implementation of the event loop in Cython.

That is what is meant by the term “pluggable event loop”: you can use any working implementation of an event loop, unrelated to the structure of the coroutines themselves. The asyncio package itself ships with two different event loop implementations, with the default being based on the selectors module. (The second implementation is built for Windows only.)

https://github.com/MagicStack/uvloop

uvloop is a fast, drop-in replacement of the built-in asyncio event loop. uvloop is implemented in Cython and uses libuv under the hood. uvloop makes asyncio 2-4x faster.

uvloop requires Python 3.7 or greater and is available on PyPI. Use pip to install it:

    pip install uvloop

Call `uvloop.install()` before calling `asyncio.run()` or manually creating an asyncio event loop:

```python
import asyncio
import uvloop

async def main():
    # Main entry-point.
    ...

uvloop.install()
asyncio.run(main())
```

## Example 3: Probing Domains

Imagine you are about to start a new blog on Python, and you plan to register a domain using a Python keyword and the .DEV suffix—for example: AWAIT.DEV. Example 21-1 is a script using asyncio to check several domains concurrently.

Note that the domains appear unordered. If you run the script, you’ll see them displayed one after the other, with varying delays. The + sign indicates your machine was able to resolve the domain via DNS. Otherwise, the domain did not resolve and may be available.

In blogdom.py, the DNS probing is done via native coroutine objects. Because the asynchronous operations are interleaved, the time needed to check the 18 domains is much less than checking them sequentially. In fact, the total time is practically the same as the time for the single slowest DNS response, instead of the sum of the times of all responses.

> `coroutine loop.getaddrinfo(host, port, *, family=0, type=0, proto=0, flags=0)` :Asynchronous version of socket.getaddrinfo().

In [ ]:
import asyncio
import socket
from keyword import kwlist

# Set maximum length of keyword for domains, because shorter is better.
MAX_KEYWORD_LEN = 4

# probe returns a tuple with the domain name and a boolean; True means the domain resolved. 
# Returning the domain name will make it easier to display the results.
async def probe(domain: str) -> tuple[str, bool]:
    # Get a reference to the asyncio event loop, so we can use it next
    loop = asyncio.get_running_loop()
    try:
        # The loop.getaddrinfo(…) coroutine-method returns a five-part tuple of parameters to connect to the given 
        # address using a socket. In this example, we don’t need the result. If we got it, the domain resolves; 
        # otherwise, it doesn’t.
        await loop.getaddrinfo(domain, None)
    except socket.gaierror:
        return (domain, False)
    return (domain, True)


async def main() -> None:
    # Generator to yield Python keywords with length up to MAX_KEYWORD_LEN
    names = (kw for kw in kwlist if len(kw) <= MAX_KEYWORD_LEN)
    # Generator to yield domain names with the .dev suffix.
    domains = (f"{name}.dev".lower() for name in names)
    # Build a list of coroutine objects by invoking the probe coroutine with each domain argument.
    coros = [probe(domain) for domain in domains]
    # asyncio.as_completed is a generator that yields coroutines that return the results of the coroutines passed to 
    # it in the order they are completed—not the order they were submitted. 
    for coro in asyncio.as_completed(coros):
        # At this point, we know the coroutine is done because that’s how as_completed works. 
        # Therefore, the await expression will not block but we need it to get the result from coro. 
        # If coro raised an unhandled exception, it would be re-raised here.
        domain, found = await coro
        mark = "+" if found else " "
        print(f"{mark} {domain}")

# asyncio.run starts the event loop and returns only when the event loop exits. 
# This is a common pattern for scripts that use asyncio: implement main as a coroutine, 
# and drive it with asyncio.run inside the if __name__ == '__main__': block.
if __name__ == "__main__":
    asyncio.run(main())

> The `asyncio.get_running_loop` function was added in Python 3.7 for use inside coroutines, as shown in probe. If there’s no running loop, `asyncio.get_running_loop` raises RuntimeError. Its implementation is simpler and faster than `asyncio.get_event_loop`, which may start an event loop if necessary. Since Python 3.10, `asyncio.get_event_loop` is deprecated and will eventually become an alias to `asyncio.get_running_loop`.

> V primeru da DNS neha delat zaženi: `sudo netplan apply`

### Trick to Read Asynchronous Code

There are a lot of new concepts to grasp in asyncio, but the overall logic of Example 21-1 is easy to follow if you employ a trick suggested by Guido van Rossum himself: squint and pretend the async and await keywords are not there. If you do that, you’ll realize that coroutines read like plain old sequential functions.

For example, imagine that the body of this coroutine:

In [ ]:
async def probe(domain: str) -> tuple[str, bool]:
    loop = asyncio.get_running_loop()
    try:
        await loop.getaddrinfo(domain, None)
    except socket.gaierror:
        return (domain, False)
    return (domain, True)

…works like the following function, except that it magically never blocks:

In [ ]:
def probe(domain: str) -> tuple[str, bool]:  # no async
    loop = asyncio.get_running_loop()
    try:
        loop.getaddrinfo(domain, None)  # no await
    except socket.gaierror:
        return (domain, False)
    return (domain, True)

Using the syntax await `loop.getaddrinfo(...)` avoids blocking because await suspends the current coroutine object. For example, during the execution of the probe('if.dev') coroutine, a new coroutine object is created by getaddrinfo('if.dev', None). Awaiting it starts the low-level addrinfo query and yields control back to the event loop, not to the probe(‘if.dev’) coroutine, which is suspended. The event loop can then drive other pending coroutine objects, such as probe('or.dev').

When the event loop gets a response for the getaddrinfo('if.dev', None) query, that specific coroutine object resumes and returns control back to the probe('if.dev')—which was suspended at await—and can now handle a possible exception and return the result tuple.

So far, we’ve only seen `asyncio.as_completed` and `await` applied to coroutines. But they handle any awaitable object. That concept is explained next.

## Awaitable

The `for` keyword works with iterables. The `await` keyword works with awaitables.

As an end user of asyncio, these are the awaitables you will see on a daily basis:
- A native coroutine object, which you get by calling a native coroutine function
- An `asyncio.Task`, which you usually get by passing a coroutine object to `asyncio.create_task()`

However, end-user code does not always need to await on a Task. We use `asyncio.create_task(one_coro())` to schedule one_coro for concurrent execution, without waiting for its return. If you **don’t expect to cancel the task or wait for it, there is no need to keep the Task object returned from `create_task`.** 

**Creating the task is enough to schedule the coroutine to run.**

In contrast, we use `await other_coro()` to **run other_coro right now and wait for its completion because we need its result before we can proceed.**

When implementing asynchronous libraries or contributing to asyncio itself, you may also deal with these lower-level awaitables:
- An object with an `__await__` method that returns an iterator; for example, an `asyncio.Future` instance (asyncio.Task is a subclass of asyncio.Future)
- Objects written in other languages using the Python/C API with a `tp_as_async.am_await` function, returning an iterator (similar to `__await__` method)

Existing codebases may also have one additional kind of awaitable: generator-based coroutine objects—which are in the process of being deprecated.

> PEP 492 states that the await expression “uses the yield from implementation with an extra step of validating its argument” and “await only accepts an awaitable.” The PEP does not explain that implementation in detail, but refers to PEP 380, which introduced yield from.

## Example 4: Downloading with asyncio and HTTPX

The flags_asyncio.py script downloads a fixed set of 20 flags from fluentpython.com. We first mentioned it in “Concurrent Web Downloads”, but now we’ll study it in detail, applying the concepts we just saw.

As of Python 3.10, asyncio only supports TCP and UDP directly, and there are no asynchronous HTTP client or server packages in the standard library. I am using HTTPX in all the HTTP client examples.

We’ll explore flags_asyncio.py from the bottom up—that is, looking first at the functions that set up the action in Example 21-2.

In [ ]:
import asyncio
import time
from pathlib import Path
from typing import Callable
# httpx must be installed—it’s not in the standard library
from httpx import AsyncClient

POP20_CC = "CN IN US ID BR PK NG BD RU JP MX PH VN ET EG DE IR TR CD FR".split()

BASE_URL = "https://www.fluentpython.com/data/flags"
DEST_DIR = Path("downloaded")

# download_one must be a native coroutine, so it can await on get_flag—which does the HTTP request
async def download_one(client: AsyncClient, cc: str):
    image = await get_flag(client, cc)
    save_flag(image, f"{cc}.gif")
    print(cc, end=" ", flush=True)
    return cc

# Then it displays the code of the downloaded flag, and saves the image.
# get_flag needs to receive the AsyncClient to make the request.
async def get_flag(client: AsyncClient, cc: str) -> bytes:
    url = f"{BASE_URL}/{cc}/{cc}.gif".lower()
    # The get method of an httpx.AsyncClient instance returns a ClientResponse object 
    # that is also an asynchronous context manager.
    resp = await client.get(url, timeout=6.1, follow_redirects=True)
    # Network I/O operations are implemented as coroutine methods, so they are 
    # driven asynchronously by the asyncio event loop.
    return resp.read()

# this needs to be a plain function—not a coroutine—so it can be passed to and called by the main function from 
# the flags.py module (Example 20-2).
def download_many(cc_list: list[str]) -> int:
    # Execute the event loop driving the supervisor(cc_list) coroutine object until it returns. 
    # This will block while the event loop runs. The result of this line is whatever supervisor returns.
    return asyncio.run(supervisor(cc_list))

# For better performance, the save_flag call inside get_flag should be asynchronous, to avoid blocking the event loop. 
# However, asyncio does not provide an asynchronous filesystem API at this time
def save_flag(img: bytes, filename: str) -> None:
    (DEST_DIR / filename).write_bytes(img)


async def supervisor(cc_list: list[str]) -> int:
    # Asynchronous HTTP client operations in httpx are methods of AsyncClient, which is also an 
    # asynchronous context manager: a context manager with asynchronous setup and teardown methods 
    async with AsyncClient() as client:
        # Build a list of coroutine objects by calling the download_one coroutine once for each flag to be retrieved.
        to_do = [download_one(client, cc) for cc in sorted(cc_list)]
        # Wait for the asyncio.gather coroutine, which accepts one or more awaitable arguments and 
        # waits for all of them to complete, returning a list of results for the given awaitables in 
        # the order they were submitted. 
        res = await asyncio.gather(*to_do)
    # supervisor returns the length of the list returned by asyncio.gather.
    return len(res)


def main(downloader: Callable[[list[str]], int]) -> None:
    DEST_DIR.mkdir(exist_ok=True)
    t0 = time.perf_counter()
    count = downloader(POP20_CC)
    elapsed = time.perf_counter() - t0
    print(f"\n{count} downloads in {elapsed:.2f}s")


if __name__ == "__main__":
    main(download_many)

Your code delegates to the httpx coroutines explicitly through await or implicitly through the special methods of the asynchronous context managers, such as Async​Client and ClientResponse—as we’ll see in “Asynchronous Context Managers”.

A key difference between the classic coroutine examples we saw in “Classic Coroutines” and flags_asyncio.py is that there are no visible .send() calls or yield expressions in the latter. Your code sits between the asyncio library and the asynchronous libraries you are using, such as HTTPX.

Under the hood, the **asyncio event loop makes the .send calls that drive your coroutines, and your coroutines await on other coroutines, including library coroutines**. As mentioned, await borrows most of its implementation from yield from, which also makes .send calls to drive coroutines.

The await chain eventually reaches a low-level awaitable, which returns a generator that the event loop can drive in response to events such as timers or network I/O. The low-level awaitables and generators at the end of these await chains are implemented deep into the libraries, are not part of their APIs, and may be Python/C extensions.

Using functions like asyncio.gather and asyncio.create_task, you can start multiple concurrent await channels, enabling concurrent execution of multiple I/O operations driven by a single event loop, in a single thread.

Note that in Example 21-3, I could not reuse the get_flag function from flags.py (Example 20-2). I had to rewrite it as a coroutine to use the asynchronous API of HTTPX. **For peak performance with asyncio, we must replace every function that does I/O with an asynchronous version that is activated with await or asyncio.create_task, so that control is given back to the event loop while the function waits for I/O.** 

**If you can’t rewrite a blocking function as a coroutine, you should run it in a separate thread or process, as we’ll see in “Delegating Tasks to Executors”.**

That’s why I chose the epigraph for this chapter, which includes this advice: “You rewrite all your code so none of it blocks or you’re just wasting your time.”

For the same reason, I could not reuse the download_one function from flags_threadpool.py (Example 20-3) either. The code in Example 21-3 drives get_flag with await, so download_one must also be a coroutine. For each request, a download_one coroutine object is created in supervisor, and they are all driven by the asyncio.gather coroutine.


## Asynchronous context manager

In “Context Managers and with Blocks”, we saw how an object can be used to run code before and after the body of a with block, if its class provides the `__enter__` and `__exit__` methods.

Now, consider Example 21-4, from the asyncpg asyncio-compatible PostgreSQL driver documentation on transactions.

In [ ]:
tr = connection.transaction()
await tr.start()
try:
    await connection.execute("INSERT INTO mytable VALUES (1, 2, 3)")
except:
    await tr.rollback()
    raise
else:
    await tr.commit()

A database transaction is a natural fit for the context manager protocol: the transaction has to be started, data is changed with connection.execute, and then a rollback or commit must happen, depending on the outcome of the changes.

In an asynchronous driver like asyncpg, the setup and wrap-up need to be coroutines so that other operations can happen concurrently. However, the implementation of the classic with statement doesn’t support coroutines doing the work of `__enter__` or `__exit__`.

That’s why PEP 492—Coroutines with async and await syntax introduced the async with statement, which works with asynchronous context managers: objects implementing the `__aenter__` and `__aexit__` methods as coroutines.

With async with, Example 21-4 can be written like this other snippet from the asyncpg documentation:

In [ ]:
async with connection.transaction():
    await connection.execute("INSERT INTO mytable VALUES (1, 2, 3)")

In the `asyncpg.Transaction` class, the `__aenter__` coroutine method does await self.start(), and the `__aexit__` coroutine awaits on private `__rollback` or `__commit` coroutine methods, depending on whether an exception occurred or not. Using coroutines to implement Transaction as an asynchronous context manager allows asyncpg to handle many transactions concurrently.

Primer:

In [ ]:
import aiohttp
import asyncio

class AsyncSession:
    def __init__(self, url):
        self._url = url

    async def __aenter__(self):
        self.session = aiohttp.ClientSession()
        response = await self.session.get(self._url)
        return response

    async def __aexit__(self, exc_type, exc_value, exc_tb):
        await self.session.close()

async def check(url):
    async with AsyncSession(url) as response:
        print(f"{url}: status -> {response.status}")
        html = await response.text()
        print(f"{url}: type -> {html[:17].strip()}")

async def main():
    await asyncio.gather(
        check("https://realpython.com"),
        check("https://pycoders.com"),
    )

asyncio.run(main())

In `.__aenter__()`, you create an aiohttp.ClientSession(), await the .get() response, and finally return the response itself. In `.__aexit__()`, you close the session, which corresponds to the teardown logic in this specific case. Note that `.__aenter__()` and `.__aexit__()` must return awaitable objects. In other words, you must define them with async def, which returns a coroutine object that is awaitable by definition.

## Using asyncio.as_completed and a Thread

> `coroutine asyncio.to_thread(func, /, *args, **kwargs)`: Asynchronously run function func in a separate thread.

Any `*args` and `**kwargs` supplied for this function are directly passed to func. Also, the current contextvars.Context is propagated, allowing context variables from the event loop thread to be accessed in the separate thread.

Return a coroutine that can be awaited to get the eventual result of func.

This coroutine function is primarily intended to be used for executing IO-bound functions/methods that would otherwise block the event loop if they were ran in the main thread. For example:

In [ ]:
import asyncio
import time


def blocking_io():
    print(f"start blocking_io at {time.strftime('%X')}")
    # Note that time.sleep() can be replaced with any blocking
    # IO-bound operation, such as file operations.
    time.sleep(1)
    print(f"blocking_io complete at {time.strftime('%X')}")


async def main():
    print(f"started main at {time.strftime('%X')}")

    await asyncio.gather(asyncio.to_thread(blocking_io), asyncio.sleep(1))

    print(f"finished main at {time.strftime('%X')}")


asyncio.run(main())


Directly calling blocking_io() in any coroutine would block the event loop for its duration, resulting in an additional 1 second of run time. Instead, by using asyncio.to_thread(), we can run it in a separate thread without blocking the event loop.

>  Due to the GIL, asyncio.to_thread() can typically only be used to make IO-bound functions non-blocking. However, for extension modules that release the GIL or alternative Python implementations that don’t have one, asyncio.to_thread() can also be used for CPU-bound functions.

In [ ]:
# 18_flags2_asyncio.py - change

async def download_one(client: AsyncClient, cc: str):
    image = await get_flag(client, cc)
    # Using asyncio.as_completed and a Thread
    await asyncio.to_thread(save_flag, image, f"{cc}.gif")
    print(cc, end=" ", flush=True)
    return cc

All network I/O is done with coroutines in asyncio, but not file I/O. However, file I/O is also “blocking”—in the sense that reading/writing files takes thousands of times longer than reading/writing to RAM. If you’re using Network-Attached Storage, it may even involve network I/O under the covers.

Since Python 3.9, the `asyncio.to_thread` coroutine makes it easy to delegate file I/O to a thread pool provided by asyncio. 

Dodamo še uvloop:

In [ ]:
import uvloop

def download_many(cc_list: list[str]) -> int:
    uvloop.install()
    return asyncio.run(supervisor(cc_list))

## Asynchronous Iteration and Asynchronous Iterables

We saw in “Asynchronous Context Managers” how async with works with objects implementing the `__aenter__` and `__aexit__` methods returning awaitables—usually in the form of coroutine objects.

Similarly, async for works with asynchronous iterables: objects that implement `__aiter__`. However, `__aiter__` must be a regular method—not a coroutine method—and it **must return an asynchronous iterator**.

An asynchronous iterator provides an `__anext__` coroutine method that returns an awaitable—often a coroutine object. They are also expected to implement `__aiter__`, which usually returns self. This mirrors the important distinction of iterables and iterators we discussed in “Don’t Make the Iterable an Iterator for Itself”.

The aiopg asynchronous PostgreSQL driver documentation has an example that illustrates the use of async for to iterate over the rows of a database cursor:

In [ ]:
async def go():
    pool = await aiopg.create_pool(dsn)
    async with pool.acquire() as conn:
        async with conn.cursor() as cur:
            await cur.execute("SELECT 1")
            ret = []
            async for row in cur:
                ret.append(row)
            assert ret == [(1,)]

In this example the query will return a single row, but in a realistic scenario you may have thousands of rows in response to a SELECT query. For large responses, **the cursor will not be loaded with all the rows in a single batch**. Therefore it is important that async for row in cur: **does not block the event loop while the cursor may be waiting for additional rows**. By implementing the cursor as an asynchronous iterator, **aiopg may yield to the event loop at each `__anext__` call**, and resume later when more rows arrive from PostgreSQL.

### Asynchronous Generator Functions

You can implement an asynchronous iterator by writing a class with `__anext__` and `__aiter__`, but there is a simpler way: write a function declared with async def and use yield in its body. This parallels how generator functions simplify the classic Iterator pattern.

Let’s study a simple example using async for and implementing an asynchronous generator. In Example 21-1 we saw blogdom.py, a script that probed domain names. Now suppose we find other uses for the probe coroutine we defined there, and decide to put it into a new module—domainlib.py—together with a new multi_probe asynchronous generator that takes a list of domain names and yields results as they are probed.



In [ ]:
# domainlib.py
import asyncio
import socket
import sys
from collections.abc import AsyncIterator, Iterable
from keyword import kwlist
from typing import NamedTuple

# NamedTuple makes the result from probe easier to read and debug.
class Result(NamedTuple):
    domain: str
    found: bool


# probe now gets an optional loop argument, to avoid repeated calls to get_running_loop when 
# this coroutine is driven by multi_probe.
async def probe(domain: str, loop: asyncio.AbstractEventLoop = None) -> Result:
    if loop is None:
        loop = asyncio.get_running_loop()
    try:
        await loop.getaddrinfo(domain, None)
    except socket.gaierror:
        return Result(domain, False)
    return Result(domain, True)

# An asynchronous generator function produces an asynchronous generator object, which can be annotated 
# as AsyncIterator[SomeType].
async def multi_probe(domains: Iterable[str]) -> AsyncIterator[Result]:
    loop = asyncio.get_running_loop()
    # Build list of probe coroutine objects, each with a different domain.
    coros = [probe(domain, loop) for domain in domains]
    # This is not async for because asyncio.as_completed is a classic generator.
    for coro in asyncio.as_completed(coros):
        # Await on the coroutine object to retrieve the result.
        result = await coro
        # Yield result. This line makes multi_probe an asynchronous generator.
        yield result


async def main(tld: str) -> None:
    tld = tld.strip(".")
    # Generate keywords with length up to 4.
    names = (kw for kw in kwlist if len(kw) <= 4)
    # Generate domain names with the given suffix as TLD.
    domains = (f"{name}.{tld}".lower() for name in names)
    #Format a header for the tabular output.
    print("FOUND\t\tNOT FOUND")
    print("=====\t\t=========")
    # Asynchronously iterate over multi_probe(domains).
    async for domain, found in multi_probe(domains):
        # set indent to zero or two tabs to put the result in the proper column.
        indent = "" if found else "\t\t"
        print(f"{indent}{domain}")


if __name__ == "__main__":
    if len(sys.argv) == 2:
        # Run the main coroutine with the given command-line argument.
        asyncio.run(main(sys.argv[1]))  # <6>
    else:
        print("Please provide a TLD.", f"Example: {sys.argv[0]} COM.BR")


Given domainlib.py, we can demonstrate the use of the multi_probe asynchronous generator in domaincheck.py: a script that takes a domain suffix and searches for domains made from short Python keywords.

**Asynchronous generators versus native coroutines**

Here are some key similarities and differences between a native coroutine and an asynchronous generator function:
- Both are declared with async def.
- An asynchronous generator always has a **yield expression** in its body—that’s what makes it a generator. A native coroutine never contains yield.
- A native coroutine may return some value other than None. An **asynchronous generator can only use empty return statements**.
- Native coroutines are awaitable: they can be driven by await expressions or passed to one of the many asyncio functions that take awaitable arguments, such as create_task. **Asynchronous generators are not awaitable**. They are asynchronous iterables, driven by `async for` or by asynchronous comprehensions.


### Async Comprehensions and Async Generator Expressions

PEP 530—Asynchronous Comprehensions introduced the use of async for and await in the syntax of comprehensions and generator expressions, starting with Python 3.6.

The only construct defined by PEP 530 that can **appear outside an async def body is an asynchronous generator expression.**

Since Python 3.8, you can run the interpreter with the `-m asyncio` command-line option to get an “async REPL”: a Python console that imports asyncio, provides a running event loop, and accepts await, async for, and async with at the top-level prompt—which otherwise are syntax errors when used outside of native coroutines.

Then run: `python -m asyncio`

Note how the header says you can use await instead of asyncio.run()—to drive coroutines and other awaitables. Also: I did not type import asyncio. The asyncio module is automatically imported and that line makes that fact clear to the user.

Given the multi_probe asynchronous generator from Example 21-18, we could write another asynchronous generator returning only the names of the domains found. Here is how—again using the asynchronous console launched with -m asyncio:

```python
>>> from domainlib import multi_probe
>>> names = 'python.org rust-lang.org golang.org no-lang.invalid'.split()
>>> gen_found = (name async for name, found in multi_probe(names) if found)  
>>> gen_found
# The asynchronous generator expression builds an async_generator object—exactly the same type of object returned by an asynchronous generator function like multi_probe.
<async_generator object <genexpr> at 0x10a8f9700>  
# The asynchronous generator expression builds an async_generator object—exactly the same type of object returned by an asynchronous generator function like multi_probe.
>>> async for name in gen_found:  
...     print(name)
```

The use of async for makes this an asynchronous generator expression. It can be defined anywhere in a Python module.

To summarize: an asynchronous generator expression can be defined anywhere in your program, but it can only be consumed inside a native coroutine or asynchronous generator function.

The remaining constructs introduced by PEP 530 can only be defined and used inside native coroutines or asynchronous generator functions.

#### Asynchronous comprehensions

Yury Selivanov—the author of PEP 530—justifies the need for asynchronous comprehensions with three short code snippets reproduced next.

We can all agree that we should be able to rewrite this code:

In [ ]:
result = []
async for i in aiter():
    if i % 2:
        result.append(i)

like this:

In [ ]:
result = [i async for i in aiter() if i % 2]

In addition, given a native coroutine fun, we should be able to write this:

In [ ]:
result = [await fun() for fun in funcs]

> Using await in a list comprehension is similar to using asyncio.gather. But gather gives you more control over exception handling, thanks to its optional return_exceptions argument. Caleb Hattingh recommends always setting return_exceptions=True (the default is False).

Back to the magic asynchronous console:

```python
>>> from domainlib import probe
>>> names = 'python.org rust-lang.org golang.org no-lang.invalid'.split()
>>> names = sorted(names)
>>> coros = [probe(name) for name in names]
>>> await asyncio.gather(*coros)
[Result(domain='golang.org', found=True),
Result(domain='no-lang.invalid', found=False),
Result(domain='python.org', found=True),
Result(domain='rust-lang.org', found=True)]
>>> [await probe(name) for name in names]
[Result(domain='golang.org', found=True),
 Result(domain='no-lang.invalid', found=False),
Result(domain='python.org', found=True),
Result(domain='rust-lang.org', found=True)]
>>>
```

Note that I sorted the list of names to show that the results come out in the order they were submitted, in both cases.

PEP 530 allows the use of async for and await in list comprehensions as well as in dict and set comprehensions. For example, here is a dict comprehension to store the results of multi_probe in the asynchronous console:

```python
>>> {name: found async for name, found in multi_probe(names)}
{'golang.org': True, 'python.org': True, 'no-lang.invalid': False,
'rust-lang.org': True}
```

We can use the await keyword in the expression before the for or async for clause, and also in the expression after the if clause. Here is a set comprehension in the asynchronous console, collecting only the domains that were found:



```python
>>> {name for name in names if (await probe(name)).found}
{'rust-lang.org', 'python.org', 'golang.org'}
```

I had to put extra parentheses around the await expression due to the higher precedence of the `__getattr__` operator . (dot).

Again, all of these comprehensions can only appear inside an async def body or in the enchanted asynchronous console.


## Async IO in Context

### When and Why Is Async IO the Right Choice?

Async IO shines when you have multiple IO-bound tasks where the tasks would otherwise be dominated by blocking IO-bound wait time, such as:
- Network IO, whether your program is the server or the client side
- Serverless designs, such as a peer-to-peer, multi-user network like a group chatroom
- Read/write operations where you want to mimic a “fire-and-forget” style but worry less about holding a lock on whatever you’re reading and writing to

The biggest reason not to use it is that await only supports a specific set of objects that define a specific set of methods. If you want to do async read operations with a certain DBMS, you’ll need to find not just a Python wrapper for that DBMS, but one that supports the async/await syntax. Coroutines that contain synchronous calls block other coroutines and tasks from running.


### Python Version Specifics

Async IO in Python has evolved swiftly, and it can be hard to keep track of what came when. Here’s a list of Python minor-version changes and introductions related to asyncio:
- 3.3: The yield from expression allows for generator delegation.
- 3.4: asyncio was introduced in the Python standard library with provisional API status.
- 3.5: async and await became a part of the Python grammar, used to signify and wait on coroutines. They were not yet reserved keywords. (You could still define functions or variables named async and await.)
- 3.6: Asynchronous generators and asynchronous comprehensions were introduced. The API of asyncio was declared stable rather than provisional.
- 3.7: async and await became reserved keywords. (They cannot be used as identifiers.) They are intended to replace the asyncio.coroutine() decorator. asyncio.run() was introduced to the asyncio package, among a bunch of other features.

If you want to be safe (and be able to use asyncio.run()), go with Python 3.7 or above to get the full set of features.

### Libraries That Work With async/await

<section class="section3" id="libraries-that-work-with-asyncawait">
<p>From <a href="https://github.com/aio-libs">aio-libs</a>:</p>
<ul>
<li><a href="https://github.com/aio-libs/aiohttp"><code>aiohttp</code></a>: Asynchronous HTTP client/server framework</li>
<li><a href="https://github.com/aio-libs/aioredis"><code>aioredis</code></a>: Async IO Redis support</li>
<li><a href="https://github.com/aio-libs/aiopg"><code>aiopg</code></a>: Async IO PostgreSQL support</li>
<li><a href="https://github.com/aio-libs/aiomcache"><code>aiomcache</code></a>: Async IO memcached client</li>
<li><a href="https://github.com/aio-libs/aiokafka"><code>aiokafka</code></a>: Async IO Kafka client</li>
<li><a href="https://github.com/aio-libs/aiozmq"><code>aiozmq</code></a>: Async IO ZeroMQ support</li>
<li><a href="https://github.com/aio-libs/aiojobs"><code>aiojobs</code></a>: Jobs scheduler for managing background tasks</li>
<li><a href="https://github.com/aio-libs/async_lru"><code>async_lru</code></a>: Simple LRU cache for async IO</li>
</ul>
<p>From <a href="https://magic.io/">magicstack</a>:</p>
<ul>
<li><a href="https://github.com/MagicStack/uvloop"><code>uvloop</code></a>: Ultra fast async IO event loop</li>
<li><a href="https://github.com/MagicStack/asyncpg"><code>asyncpg</code></a>: (Also very fast) async IO PostgreSQL support</li>
</ul>
<p>From other hosts:</p>
<ul>
<li><a href="https://github.com/python-trio/trio"><code>trio</code></a>: Friendlier <code>asyncio</code> intended to showcase a radically simpler design</li>
<li><a href="https://github.com/Tinche/aiofiles"><code>aiofiles</code></a>: Async file IO</li>
<li><a href="https://github.com/theelous3/asks"><code>asks</code></a>: Async requests-like http library</li>
<li><a href="https://github.com/jonathanslenders/asyncio-redis"><code>asyncio-redis</code></a>: Async IO Redis support</li>
<li><a href="https://github.com/dano/aioprocessing"><code>aioprocessing</code></a>: Integrates <code>multiprocessing</code> module with <code>asyncio</code></li>
<li><a href="https://github.com/Scille/umongo"><code>umongo</code></a>: Async IO MongoDB client</li>
<li><a href="https://github.com/alex-sherman/unsync"><code>unsync</code></a>: Unsynchronize <code>asyncio</code></li>
<li><a href="https://github.com/vxgmichel/aiostream"><code>aiostream</code></a>: Like <code>itertools</code>, but async</li>
</ul>
</section>

Kaj narediti v primeru ko knjižnice nimajo podpore za asyincio? Uporabi threade če je možno.